# 多体の体積を大きくしていった時に収束するエネルギーをQEと機械学習で比べる
- https://www.notion.so/box-QE-d509d0092a2e4d0c96d2dca70e990dd1?pvs=4

In [1]:
from ase.io.vasp import read_vasp
from ase.build import make_supercell

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlptools.io.write import QuantumEspressoWriter
import os
import pickle

from mlptools.io.write import N2p2Writer
from glob import glob

In [6]:
path2structure = "/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/many-body-analysis/structure/POSCAR"

atoms = read_vasp(path2structure)

In [7]:
# make supercell
supercell_atoms = make_supercell(atoms, [[2, 0, 0], [0, 2, 0], [0, 0, 2]])
print(len(supercell_atoms))

64


In [29]:
expanded_atoms_list = []

# change volume
change_volume_list = np.linspace(0.5, 3.00, 26)
print(change_volume_list)
for change_volume in change_volume_list:
    expanded_atoms = supercell_atoms.copy()
    expanded_atoms.set_cell(expanded_atoms.get_cell() * change_volume, scale_atoms=True)
    expanded_atoms_list.append(expanded_atoms)

In [33]:
# 原子間距離を計算して、最小値を取得する。しかし書く行に０があるのでそれを除いた最小値
min_distance_list = []
for expanded_atoms in expanded_atoms_list:
    distance_matrix = expanded_atoms.get_all_distances()
    np.fill_diagonal(distance_matrix, np.inf)
    min_distance_each_atoms = np.min(distance_matrix, axis=1)
    # 全て同じ値かどうか
    if np.all(min_distance_each_atoms == min_distance_each_atoms[0]):
        print("All atoms are same distance.")
    else:
        print("All atoms are not same distance.")
    
    min_distance_list.append(np.min(min_distance_each_atoms))

All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.
All atoms are not same distance.


In [40]:
# 計算できるようにディレクトリを作成する
path2output = "/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/many-body-analysis/output"
path2template = "/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/many-body-analysis/template"

for i, expanded_atoms in enumerate(expanded_atoms_list):
    writer = QuantumEspressoWriter(
        atoms=expanded_atoms,
        path2template=path2template,
    )
    scf_input_lines = writer.output()

    dir_name = f"scf_{i}"
    path2dir = os.path.join(path2output, dir_name)
    os.makedirs(path2dir, exist_ok=True)

    with open(os.path.join(path2dir, "scf.in"), mode="w") as f:
        f.writelines('\n'.join(scf_input_lines))
    
    # save atoms as pickle
    pickle.dump(expanded_atoms, open(os.path.join(path2dir, "atoms.pkl"), mode="wb"))

In [3]:
# n2p2用のinputを作成する

path2output = "/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/many-body-analysis/output"
path2n2p2_output = "/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/many-body-analysis/n2p2_output"

all_atoms = []
scf_dirs = glob(os.path.join(path2output, "scf_*"))
for scf_dir in scf_dirs:
    atoms = pickle.load(open(os.path.join(scf_dir, "atoms.pkl"), mode="rb"))
    writer = N2p2Writer(
        atoms=atoms,
        is_comment=True,
        structure_id=os.path.basename(scf_dir),
        has_calculator=False,
    )
    path2target = os.path.join(path2n2p2_output, os.path.basename(scf_dir))
    os.makedirs(path2target, exist_ok=True)
    with open(os.path.join(path2target, "input.data"), mode="w") as f:
            f.write("\n".join(writer.output()))

## 結果確認

In [2]:
path2n2p2_output = "/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/many-body-analysis/n2p2_output/model_21"
path2scf = "/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/many-body-analysis/output"

ml_pred_dir_list = glob(os.path.join(path2n2p2_output, "scf_*"))
scf_dir_list = glob(os.path.join(path2scf, "scf_*"))
print(len(ml_pred_dir_list), len(scf_dir_list))

26 26


In [3]:
from mlptools.io.read import read_from_n2p2_data

for ml_pred_dir in ml_pred_dir_list:
    mlpatoms_list = read_from_n2p2_data(ml_pred_dir, "output.data")    
    break

0 / 1
